Gen List

In [17]:
import pandas as pd
df = pd.read_excel('data/Refrence/2023_celt_report.xlsx', skiprows=13, index_col=None, sheet_name='2.1 Generator List')
df2 = df[['ASSET ID', 'ASSET NAME', 'RESOURCE ID', 'RESOURCE NAME',
       'GENERATOR TYPE', 'PRIM FUEL TYPE', 'ALT FUEL TYPE', 'EIA PLANT NUMBER', 'NAMEPLATE\n(MW)']]
df2.rename(columns={"NAMEPLATE\n(MW)": "Nameplate Capacity (MW)", "PRIM FUEL TYPE": "Primary Fuel Type"}, inplace=True)

from utilsData import fuelDict

fuels = df2['Primary Fuel Type'].map(fuelDict)
# To add Pump Storage
# if True:
#     fuels.loc[(df2['Primary Fuel Type'] == 'WAT') & (df2['GENERATOR TYPE'] == 'HYDRO: PUMPED STORAGE')] = 'PS'

fuels = fuels.fillna('Other')
df2.insert(3, 'Fuel Type', fuels)

/tmp/ipykernel_6605/3841821998.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={"NAMEPLATE\n(MW)": "Nameplate Capacity (MW)", "PRIM FUEL TYPE": "Primary Fuel Type"}, inplace=True)


In [18]:
df2.to_csv('data/CELT2023.csv', index=False)

CSO

In [19]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime
# from main import getHourlyGen, getHourlyLoad

In [20]:
df = pd.read_excel('data/Refrence/fca_obligations.xlsx', skiprows=0, index_col=None, sheet_name='FCA 14')
df = df.drop(['Intermittent', 'DR Type', 'Active/ Passive', 'Interface ID', 'Interface Name', 'Dispatch Zone ID', 'Dispatch Zone Name',
              'Lead Participant ID', 'Lead Participant Name', 'De-list Bid Type', 'De-list Bid MW', 'Status', 'Capacity Zone ID', 'Capacity Zone Name', 'Load Zone'], axis=1)
new_columns = pd.to_datetime(df.columns[-12:]).strftime('%B')
df.columns = df.columns[:-12].tolist() + new_columns.tolist()
df = df[df['Type'] == 'Generator']
df.reset_index(drop=True, inplace=True)


In [21]:
from utilsData import getISO
from utilsData import fuelDict
dfISO, numGenerators, totalCap, totalCSO = getISO()

Total Capacity:  38389.70230400001 Number of Generators:  2585


In [22]:
def map(x):
    # if 'biomass' in x:
    #     print(x.lower())
    x = x.lower()
    if 'water' in x or 'hydro' in x:
        return 'Hydro'
    
    elif 'wind' in x:
        return 'Wind'
    
    elif 'waste' in x or 'wood' in x:
        return 'Refuse/Woods'
    
    elif 'solar' in x or 'photo' in x:
        return 'Solar'
    
    elif 'landfill gas' in x:
        return 'Landfill Gas'
    
    elif 'biomass gas' in x or 'methane' in x:
        return 'Gas-Other'
    
    elif 'gas' in x or 'ker' in x:
        return 'Gas'
    
    elif 'coal' in x:
        return 'Coal'
    
    elif 'oil' in x or 'diesel' in x or 'jet' in x:
        return 'Oil'    
    
    elif 'nuclear' in x:
        return 'NUC'
    
    elif 'storage' in x:
        return 'ES'
    
    else:
        return 'Other'


In [23]:
# np.isin(df['ID'], dfISO['ASSET ID'])

In [24]:
fuels = df[['ID', 'Generating Fuel Type']]
fuels['Fuel Type'] = pd.Series()
for i in range(0, len(fuels['Fuel Type'])):
     a = dfISO[dfISO['ASSET ID'] == fuels.iloc[i]['ID']]['Fuel Type']
     if len(a) > 0:
          fuels['Fuel Type'][i] = a.to_list()[0]
     else:
          fuels['Fuel Type'][i] = map(fuels['Generating Fuel Type'][i])

/tmp/ipykernel_6605/642114524.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  fuels['Fuel Type'] = pd.Series()
/tmp/ipykernel_6605/642114524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuels['Fuel Type'] = pd.Series()
/tmp/ipykernel_6605/642114524.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuels['Fuel Type'][i] = a.to_list()[0]
/tmp/ipykernel_6605/642114524.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [25]:
fuels = fuels.fillna('Other')
set(fuels['Fuel Type'])

{'Coal',
 'ES',
 'Gas',
 'Gas-Other',
 'Hydro',
 'Landfill Gas',
 'Nuclear',
 'Oil',
 'Refuse/Woods',
 'Solar',
 'Wind'}

In [26]:
df.insert(3, 'Fuel Type', fuels['Fuel Type'])
df.to_csv('data/CSO2023.csv', index=False)